# https://youtu.be/vF21cC-8G1U
# https://youtu.be/Joh3LOaG8Q0

"""
Dataset from: https://lhncbc.nlm.nih.gov/publication/pub9932
Binary problem:
Question is: Is the image uninfected? If yes, probability is close to 1.
If no, the probablility is close to 0.
This is because we added label 1 to uninfected images. 
In summary, probability result close to 1 reflects uninfected image
and close to 0 reflects parasitized image
"""

In [2]:
##########################################################
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.style.use('classic')
#############################################################
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
#from keras import backend as K
####################################################
import os
import cv2
from PIL import Image
import numpy as np
import tensorflow as tf

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
tf.config.list_physical_devices('GPU')

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1498385982512822483
]


[]

In [4]:
image_directory = "/home/pratham/study/input_data/washer/da_opencv/"
SIZE = 150
dataset = []  #Many ways to handle data, you can use pandas. Here, we are using a list format.  
label = []  #Place holders to define add labels. We will add 0 to all parasitized images and 1 to uninfected.

parasitized_images = os.listdir(image_directory + 'neg/')
for i, image_name in enumerate(parasitized_images):    #Remember enumerate method adds a counter and returns the enumerate object
    
    if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory + 'neg/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(0)

In [5]:
#Iterate through all images in Uninfected folder, resize to 150 x 150
#Then save into the same numpy array 'dataset' but with label 1

uninfected_images = os.listdir(image_directory + 'pos/')
for i, image_name in enumerate(uninfected_images):
    if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory + 'pos/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(1)

In [6]:
dataset = np.array(dataset)
label = np.array(label)

In [7]:
dataset.shape

(305, 150, 150, 3)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset, label, test_size = 0.20, random_state = 0)

In [8]:
#Without scaling (normalize) the training may not converge. 
#Normalization is a rescaling of the data from the original range 
#so that all values are within the range of 0 and 1.
from tensorflow.keras.utils import normalize
X_train = normalize(X_train, axis=1)
X_test = normalize(X_test, axis=1)

In [12]:
###2 conv and pool layers. with some normalization and drops in between.

INPUT_SHAPE = (SIZE, SIZE, 3)   #change to (SIZE, SIZE, 3)


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=INPUT_SHAPE))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), kernel_initializer = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), kernel_initializer = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))  
#Do not use softmax for binary classification
#Softmax is useful for mutually exclusive classes, either cat or dog but not both.
#Also, softmax outputs all add to 1. So good for multi class problems where each
#class is given a probability and all add to 1. Highest one wins. 

#Sigmoid outputs probability. Can be used for non-mutually exclusive problems.
#But, also good for binary mutually exclusive (cat or not cat). 

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',             #also try adam
              metrics=['accuracy'])

print(model.summary())    
###############################################################  

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_5 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_6 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 34, 34, 64)       

In [13]:
history = model.fit(X_train, 
                         y_train, 
                         batch_size = 16, 
                         verbose = 1, 
                         epochs = 50,      
                         validation_data=(X_test,y_test),
                         shuffle = False
                     )

Epoch 1/50
16/16 [==============================] - 2s 49ms/step - loss: 0.7105 - accuracy: 0.5013 - val_loss: 0.6953 - val_accuracy: 0.4590
Epoch 2/50
16/16 [==============================] - 0s 9ms/step - loss: 0.6925 - accuracy: 0.4835 - val_loss: 0.6906 - val_accuracy: 0.4918
Epoch 3/50
16/16 [==============================] - 0s 9ms/step - loss: 0.6904 - accuracy: 0.5280 - val_loss: 0.6884 - val_accuracy: 0.4590
Epoch 4/50
16/16 [==============================] - 0s 9ms/step - loss: 0.6822 - accuracy: 0.5630 - val_loss: 0.6608 - val_accuracy: 0.5738
Epoch 5/50
16/16 [==============================] - 0s 9ms/step - loss: 0.6809 - accuracy: 0.5833 - val_loss: 0.6530 - val_accuracy: 0.6066
Epoch 6/50
16/16 [==============================] - 0s 9ms/step - loss: 0.6762 - accuracy: 0.6162 - val_loss: 0.6401 - val_accuracy: 0.6066
Epoch 7/50
16/16 [==============================] - 0s 9ms/step - loss: 0.6436 - accuracy: 0.6162 - val_loss: 0.6347 - val_accuracy: 0.5902
Epoch 8/50
16/16 [=

In [14]:
def Alexnet():
    INPUT_SHAPE = (SIZE, SIZE, 3)   #change to (SIZE, SIZE, 3)
    # Initialize the model
    model = Sequential()

    # layer 1: convolutional layer + max-pooling layer
    model.add(Conv2D(filters = 96, kernel_size = (11,11), strides= 4, padding = 'valid', activation='relu', input_shape = INPUT_SHAPE))
    model.add(MaxPooling2D(pool_size = (3,3), strides = 2))

    # layer 2: convolutional layer + max-pooling layer 
    model.add(Conv2D(filters = 256, kernel_size = (5,5), padding = 'same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (3,3), strides = 2))

    # layers 3-5: three convolutional layers + 1 max-pooling layer
    model.add(Conv2D(filters = 384, kernel_size = (3,3), padding = 'same', activation = 'relu'))
    model.add(Conv2D(filters = 384, kernel_size = (3,3), padding = 'same', activation = 'relu'))
    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = 'same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (3,3), strides = 2))

    # layers 6 - 8: two fully connected hidden layers and one fully connected output layer
    model.add(Flatten())
    model.add(Dense(4096, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation = 'sigmoid'))

    # compile the model with a loss funciton, a metric and and optimizer method for estimating the loss function
    
    model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',             #also try adam
              metrics=['accuracy'])

    return model

In [ ]:
Alexnet_model = Alexnet()
Alexnet_model.summary()

In [ ]:
history = Alexnet_model.fit(X_train, 
                         y_train, 
                         batch_size = 16, 
                         verbose = 1, 
                         epochs = 50,      
                         validation_data=(X_test,y_test),
                         shuffle = False
                     )